In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import os 

In [5]:
ls

96 Well plate templatePCR.xlsx
AquaticPig/
AquaticPig.ipynb
AustriaBodyFarm Jan2020.ipynb
AustriaBodyFarmAMS/
BNF ITS OCT 3 JPR.tiff
BenbowSubmission21GelOct2018.tiff
CourtneyLabPig/
CourtneyMaggotExp.ipynb
FROST/
FROST Jan2020.ipynb
Gel10.16.18.tif*
Gel6.24.19JR.pptx
Gel6.26.19JR.tiff
Gel7.12.19JR.tiff
GelBenbowSub21Oct2018.pptx
GelImagesFeb16SSubmission_Benbow47_48.pptx
GelPhotosAndConditionsJR.pptx
GermanySoil2020/
HippoMU/
HippoMU.ipynb
ItalyInverts/
ItalyInverts2018.ipynb
Kansas Soil Qiime2.ipynb
Kansas SoilDec2019.ipynb
KansasSoil/
KansasSoil9.27.19JR.tiff
LeafAndAMSGelPhoto.jpg
LeafOviposition/
LeafOviposition Jan2020.ipynb
MaggotLab Jan2020.ipynb
Mealworm2018/
Mealworm2018.ipynb
Mosquito oviposition antibiotic trial.docx
Purdue Pig.ipynb
PurdueMaggots/
Starling/
Starling.ipynb
Starling2018Gel.jpg
StarlingGel16SOct3JPR.tiff
ThermalCyclerAndFreezerPricing.xlsx
silva-132-99-515-806-nb-classifier.qza


In [6]:
cd CourtneyLabPig/

/Users/benbowlab/Documents/Receveur/CourtneyLabPig


In [7]:
#Import paired ends reads
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path Raw \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux.qzv


Imported Raw as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza
Saved Visualization to: demux.qzv


In [8]:
!qiime tools view demux.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [9]:
print("Start")
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-table table.qza \
  --p-trunc-len-f 250\
  --p-trunc-len-r 200\
  --o-denoising-stats denoising-stats
print("End")

Start
Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza
End


In [12]:
!qiime feature-table filter-features \
  --i-table table.qza \
  --p-min-frequency 2 \
  --o-filtered-table table.qza
!qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file MaggotExpMetadata.txt
!qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv
!qiime alignment mafft \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza
!qiime alignment mask \
  --i-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza
!qiime phylogeny fasttree \
  --i-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza
!qiime phylogeny midpoint-root \
  --i-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza
!qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 15000 \
  --m-metadata-file MaggotExpMetadata.txt \
  --o-visualization alpha-rarefaction.qzv
print("Done")

Saved FeatureTable[Frequency] to: table.qza
Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv
Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza
Saved Visualization to: alpha-rarefaction.qzv
Done


In [13]:
!qiime tools view alpha-rarefaction.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [11]:
#Classifier already built
#!qiime tools import \
#  --type 'FeatureData[Sequence]' \
#  --input-path 99_otus.fasta \
#  --output-path 99_otus.qza

#!qiime tools import \
#  --type 'FeatureData[Taxonomy]' \
#  --source-format HeaderlessTSVTaxonomyFormat \
#  --input-path 99_otu_taxonomy.txt \
#  --output-path ref-taxonomy.qza

In [20]:
#Classifier already built
#!qiime feature-classifier extract-reads \
#  --i-sequences 99_otus.qza \
#  --p-f-primer GTGCCAGCMGCCGCGGTAA \
#  --p-r-primer GGACTACHVGGGTWTCTAAT \
#  --o-reads ref-seqs.qza

Saved FeatureData[Sequence] to: ref-seqs.qza


In [14]:
#Classifier already built
#!qiime feature-classifier fit-classifier-naive-bayes \
 # --i-reference-reads ref-seqs.qza \
 # --i-reference-taxonomy ref-taxonomy.qza \
 # --o-classifier classifier.qza

Saved TaxonomicClassifier to: classifier.qza


In [14]:
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-132-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

!qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

Saved FeatureData[Taxonomy] to: taxonomy.qza
Saved Visualization to: taxonomy.qzv


In [15]:
#To fix bug with taxonomy file having spaces
!qiime tools export \
  --input-path taxonomy.qza \
  --output-path taxonomy-with-spaces/
!qiime metadata tabulate \
  --m-input-file taxonomy-with-spaces/taxonomy.tsv  \
  --o-visualization taxonomy-as-metadata.qzv
!qiime tools export \
  --input-path taxonomy-as-metadata.qzv \
  --output-path taxonomy-as-metadata
!qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-path taxonomy-as-metadata/metadata.tsv \
  --output-path taxonomy-without-spaces.qza

Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory taxonomy-with-spaces/
Saved Visualization to: taxonomy-as-metadata.qzv
Exported taxonomy-as-metadata.qzv as Visualization to directory taxonomy-as-metadata
Imported taxonomy-as-metadata/metadata.tsv as TSVTaxonomyDirectoryFormat to taxonomy-without-spaces.qza


In [16]:
!qiime taxa filter-table \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table Filteredtable.qza


Saved FeatureTable[Frequency] to: Filteredtable.qza


In [17]:

!qiime taxa barplot \
  --i-table Filteredtable.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file MaggotExpMetadata.txt \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv


In [24]:
!qiime tools view taxa-bar-plots.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [18]:
#Rarefied to 4K
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table Filteredtable.qza \
  --p-sampling-depth 4000 \
  --m-metadata-file MaggotExpMetadata.txt \
  --output-dir core-metrics-results
    
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/shannon_vector.qza \
  --m-metadata-file MaggotExpMetadata.txt \
  --o-visualization core-metrics-results/shannon-group-significance.qzv
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file MaggotExpMetadata.txt\
  --o-visualization core-metrics-results/evenness-group-significance.qzv
!qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file MaggotExpMetadata.txt \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv


Saved FeatureTable[Frequency] to: core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results/jacc

In [19]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file MaggotExpMetadata.txt \
  --m-metadata-column Treatment \
  --o-visualization core-metrics-results/weighted-unifrac-Treatment-significance.qzv \
  --p-pairwise

Saved Visualization to: core-metrics-results/weighted-unifrac-Treatment-significance.qzv


In [24]:
!qiime tools view core-metrics-results/weighted-unifrac-Treatment-significance.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [20]:
!qiime tools export \
  --input-path unrooted-tree.qza \
  --output-path exported/

Exported unrooted-tree.qza as NewickDirectoryFormat to directory exported/


In [21]:
!qiime tools export \
  --input-path core-metrics-results/rarefied_table.qza \
  --output-path exported
!qiime tools export \
  --input-path taxonomy.qza \
  --output-path exported

Exported core-metrics-results/rarefied_table.qza as BIOMV210DirFmt to directory exported
Exported taxonomy.qza as TSVTaxonomyDirectoryFormat to directory exported


Change first line of taxonomy.tsv file to #OTUID taxonomy confidence

In [22]:
!biom add-metadata -i exported/feature-table.biom -o LabMaggotMar2020.biom --observation-metadata-fp exported/taxonomy.tsv --sc-separated taxonomy

In [33]:
!biom convert -i LabMaggotMar2020.biom -o LabMaggotOTUTable.txt --to-tsv --header-key taxonomy